In [1]:
import pandas as pd
from pathlib import Path

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from allennlp.predictors.predictor import Predictor
from allennlp.models.archival import load_archive
from allennlp.common.util import lazy_groups_of
from typing import List, Iterator
from allennlp.data import Instance
from pathlib import Path
import fire

In [10]:
def predict(document_path, model_path, out_path, batch_size=4):
    def get_instance_data(document_path) -> Iterator[Instance]:
        yield from predictor._dataset_reader.read(Path(document_path))

    def predict_instances(batch_data: List[Instance]) -> Iterator[str]:
        yield predictor.predict_batch_instance(batch_data)

    # model_path = 'model_title/model.tar.gz'

    print('Loading model from %s' % model_path)
    archive = load_archive(archive_file=model_path, cuda_device=0)
    predictor = Predictor.from_archive(archive, 'text_classifier')

    count = 0

    with open(out_path, mode='w', encoding='utf-8') as out_file:
        print('Loading batches from %s for prediction' % document_path)
        out_file.write('Id,Category\n')
        idx = 0
        for batch in lazy_groups_of(get_instance_data(document_path), batch_size):
            for items, results in zip(batch, predict_instances(batch)):
                for item, result in zip(items, results):
                    print(item)
                    print(result)
                    break
                    count += 1
                    predicted_label = result['label']
                    out_file.write(str(idx) + ',' + predicted_label + '\n')
                    idx += 1
                    if count % 100 == 0:
                        print('Predicted %d sentences' % count)
    out_file.close()
    print('Finished predicting %d sentences' % count)
    print('Results saved in %s' % Path(out_path).absolute())

In [11]:
predict(document_path='test_submission.json', model_path='snli-roberta-full/model.tar.gz', out_path='submissions_snli_roberta_full.csv')

Loading model from snli-roberta-full/model.tar.gz
Loading batches from test_submission.json for prediction
tokens
{'logits': [-3.0299172401428223, 6.20608377456665, -2.782586097717285], 'probs': [9.744491399032995e-05, 0.9997777342796326, 0.00012478820281103253], 'token_ids': [0, 357, 8755, 4083, 3594, 30, 108865, 384, 296, 8543, 149322, 35, 1857, 58170, 709, 290, 3508, 41512, 622, 6911, 46323, 35, 341, 329, 88, 18, 265, 19, 81, 6683, 91, 25, 91, 2776, 48, 2], 'label': 'Negativo', 'tokens': ['<s>', 'Ġ@', 'JD', 'aniel', 'df', ':', 'ĠPedindo', 'Ġpara', 'Ġque', 'ĠMG', 'Ġreaja', '?', 'ĠRe', 'agir', 'Ġcontra', 'Ġo', 'Ġgovernador', 'Ġqueridinho', 'Ġdos', 'Ġeleitores', 'Ġmineiros', '?', 'Ġhttps', '://', 't', '.', 'co', '/', 'm', 'MC', 'w', '5', 'w', 'yy', 'L', '</s>']}
tokens
{'logits': [-3.73759388923645, 6.195363998413086, -2.2650222778320312], 'probs': [4.853536302107386e-05, 0.9997398257255554, 0.0002116352115990594], 'token_ids': [0, 357, 7934, 30, 47073, 484, 584, 8, 4071, 16, 28, 2405,

tokens
{'logits': [6.961755275726318, -2.352102041244507, -3.270143508911133], 'probs': [0.9998738765716553, 9.015472460305318e-05, 3.599878618842922e-05], 'token_ids': [0, 17317, 16, 8665, 59998, 8356, 16, 42017, 262, 10671, 356, 99610, 225, 341, 329, 88, 18, 265, 19, 13232, 59, 9919, 85, 22, 84, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 'Positivo', 'tokens': ['<s>', 'ĠdenÃºncia', ',', 'ĠPM', 'Ġapreende', 'Ġdrogas', ',', 'ĠmuniÃ§Ã£o', 'Ġe', 'Ġarmas', 'Ġem', 'ĠAraxÃ¡', 'Ġ', 'Ġhttps', '://', 't', '.', 'co', '/', 'Ox', 'W', 'HQ', 'q', '2', 'p', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']}
tokens
{'logits': [-2.7311792373657227, -2.2480459213256836, 4.862624168395996], 'probs': [0.0005028984160162508, 0.000815271632745862, 0.9986818432807922], 'token_ids': [0, 357, 7934, 30, 2384, 48985, 20869, 356, 8543, 30, 30904, 12113, 410, 6057, 384, 2851, 584, 8, 532, 16, 22, 3774, 341, 329, 88, 18, 265, 19, 25, 69, 26, 1

tokens
{'logits': [-2.4507484436035156, -3.1436004638671875, 5.665140628814697], 'probs': [0.0002986206382047385, 0.00014935439685359597, 0.9995520710945129], 'token_ids': [0, 357, 298, 67, 392, 30, 2779, 511, 5, 4305, 263, 5510, 348, 6843, 7172, 323, 2755, 274, 7103, 274, 733, 16, 13078, 274, 6166, 18, 5289, 410, 339, 341, 329, 88, 18, 265, 19, 7873, 15930, 4314, 9452, 53, 341, 329, 2], 'label': 'Neutro', 'tokens': ['<s>', 'Ġ@', 'em', '_', 'com', ':', 'ĠBom', 'Ġdia', '!', 'ĠVeja', 'Ġa', 'Ġcapa', 'Ġda', 'ĠediÃ§Ã£o', 'Ġdigital', 'Ġdo', 'ĠEstado', 'Ġde', 'ĠMinas', 'Ġde', 'Ġhoje', ',', 'Ġ09', 'Ġde', 'Ġjaneiro', '.', 'ĠLeia', 'Ġmais', 'Ġno', 'Ġhttps', '://', 't', '.', 'co', '/', 'rap', 'Yr', 'fs', 'nz', 'Q', 'Ġhttps', '://', '</s>']}
tokens
{'logits': [-3.397783041000366, 6.318267822265625, -2.6040573120117188], 'probs': [6.029599535395391e-05, 0.9998063445091248, 0.00013335193216335028], 'token_ids': [0, 357, 20775, 48516, 30, 1996, 12433, 12341, 16, 1265, 274, 8543, 2656, 1507, 305, 1449

tokens
{'logits': [-3.4884257316589355, -2.5471584796905518, 5.983168125152588], 'probs': [7.698743866058066e-05, 0.00019733629596885294, 0.9997256398200989], 'token_ids': [0, 303, 7482, 9049, 348, 8365, 9002, 263, 4071, 16, 27, 7153, 274, 15260, 356, 10290, 836, 2755, 274, 7103, 341, 329, 88, 18, 265, 19, 5419, 26175, 24, 16521, 15288, 2, 1, 1, 1], 'label': 'Neutro', 'tokens': ['<s>', 'Ã¡', 'vit', 'Ġcomercial', 'Ġda', 'ĠAlemanha', 'Ġsobe', 'Ġa', 'Ġ21', ',', '7', 'ĠbilhÃµes', 'Ġde', 'Ġeuros', 'Ġem', 'Ġnovembro', 'Ġ-', 'ĠEstado', 'Ġde', 'ĠMinas', 'Ġhttps', '://', 't', '.', 'co', '/', 'cm', 'zD', '4', 'Hp', 'Cf', '</s>', '<pad>', '<pad>', '<pad>']}
tokens
{'logits': [-2.0905210971832275, -3.0997376441955566, 5.235811233520508], 'probs': [0.0006573923164978623, 0.00023962247360032052, 0.9991029500961304], 'token_ids': [0, 3515, 4812, 960, 63879, 274, 5931, 18253, 16, 308, 3137, 1059, 341, 329, 88, 18, 265, 19, 73, 9311, 9571, 55388, 45, 29, 341, 329, 88, 18, 265, 19, 85, 1409, 33224, 5850

tokens
{'logits': [6.8851494789123535, -2.0588603019714355, -3.630154609680176], 'probs': [0.9998424053192139, 0.0001304960751440376, 2.711395063670352e-05], 'token_ids': [0, 357, 8663, 34921, 30, 1200, 3437, 997, 16, 40200, 10952, 356, 7103, 11296, 324, 1827, 20631, 5378, 18, 366, 662, 16, 9192, 35, 341, 329, 88, 18, 265, 19, 29832, 7115, 61894, 48, 2, 1, 1], 'label': 'Positivo', 'tokens': ['<s>', 'Ġ@', 'felipe', 'fonte', ':', 'ĠCom', 'ĠtrÃªs', 'Ġanos', ',', 'ĠpresÃŃdio', 'Ġprivado', 'Ġem', 'ĠMinas', 'ĠGerais', 'ĠnÃ£o', 'Ġteve', 'Ġrebel', 'iÃµes', '.', 'ĠE', 'Ġagora', ',', 'ĠJosÃ©', '?', 'Ġhttps', '://', 't', '.', 'co', '/', 'zO', 'Gi', '877', 'L', '</s>', '<pad>', '<pad>']}
tokens
{'logits': [-3.7122135162353516, -1.3880335092544556, 5.070651531219482], 'probs': [0.00015307473950088024, 0.0015641641803085804, 0.998282790184021], 'token_ids': [0, 1922, 13162, 94822, 274, 31635, 356, 21754, 274, 6355, 274, 9333, 836, 13313, 836, 2755, 274, 7103, 341, 329, 88, 18, 265, 19, 75, 6457, 135

...

tokens
{'logits': [-2.901075601577759, -2.983099937438965, 5.882585525512695], 'probs': [0.00015317107317969203, 0.00014110874326433986, 0.9997057318687439], 'token_ids': [0, 434, 51277, 16, 1688, 34037, 709, 2684, 356, 7481, 96411, 341, 329, 88, 18, 265, 19, 81, 3192, 5779, 3742, 89, 10717, 341, 329, 88, 18, 265, 19, 1332, 2915, 15533, 4607, 2, 1, 1, 1, 1, 1, 1], 'label': 'Neutro', 'tokens': ['<s>', 'Ġser', 'Ġagredida', ',', 'Ġmulher', 'Ġatira', 'Ġcontra', 'Ġhomem', 'Ġem', 'ĠGovernador', 'ĠValadares', 'Ġhttps', '://', 't', '.', 'co', '/', 'm', 'IR', 'tn', '32', 'u', 'GM', 'Ġhttps', '://', 't', '.', 'co', '/', 'ed', 'PA', 'JT', 'MG', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']}
tokens
{'logits': [-3.6507411003112793, 6.248327255249023, -2.3047478199005127], 'probs': [5.0209215260110795e-05, 0.9997568726539612, 0.00019290397176519036], 'token_ids': [0, 357, 50, 67, 73181, 30, 75253, 16, 8906, 17138, 16, 4790, 11662, 2673, 5508, 8222, 35, 203, 203, 1219, 12433, 12341, 1

tokens
{'logits': [-3.6855061054229736, -2.3502635955810547, 5.864651203155518], 'probs': [7.116572669474408e-05, 0.0002704952785279602, 0.9996583461761475], 'token_ids': [0, 357, 7159, 67, 47092, 67, 5839, 30, 3749, 16007, 584, 8, 693, 16, 24, 7153, 263, 146404, 51630, 348, 20675, 17, 39139, 836, 57088, 836, 2755, 274, 7103, 341, 329, 88, 18, 265, 19, 87, 5547, 2140, 2137, 57, 2], 'label': 'Neutro', 'tokens': ['<s>', 'Ġ@', 'Cris', '_', 'duh', '_', '123', ':', 'ĠGoverno', 'Ġpagou', 'ĠR', '$', 'Ġ2', ',', '4', 'ĠbilhÃµes', 'Ġa', 'Ġempreiteiras', 'Ġalvos', 'Ġda', 'ĠLava', '-', 'Jato', 'Ġ-', 'ĠPolitica', 'Ġ-', 'ĠEstado', 'Ġde', 'ĠMinas', 'Ġhttps', '://', 't', '.', 'co', '/', 's', 'DS', 'zz', '77', 'U', '</s>']}
tokens
{'logits': [6.963513374328613, -2.538334846496582, -2.9475560188293457], 'probs': [0.9998756647109985, 7.470432319678366e-05, 4.96161483169999e-05], 'token_ids': [0, 274, 4388, 731, 5931, 18253, 408, 2640, 356, 131704, 33122, 11142, 341, 329, 88, 18, 265, 19, 31659, 27, 7161,

tokens
{'logits': [-2.0626277923583984, -3.4894726276397705, 5.670960426330566], 'probs': [0.0004376324941404164, 0.0001050603314070031, 0.9994572997093201], 'token_ids': [0, 328, 6118, 263, 19956, 2408, 1727, 40200, 356, 7481, 96411, 341, 329, 88, 18, 265, 19, 13761, 1872, 10669, 716, 7580, 350, 38810, 32973, 350, 18543, 3414, 57681, 350, 6952, 93, 18543, 3414, 57681, 3336, 45, 2133, 2], 'label': 'Neutro', 'tokens': ['<s>', 'ĠÃ©', 'Ġmorto', 'Ġa', 'Ġtiros', 'ĠapÃ³s', 'Ġdeixar', 'ĠpresÃŃdio', 'Ġem', 'ĠGovernador', 'ĠValadares', 'Ġhttps', '://', 't', '.', 'co', '/', 'vk', 'SO', 'AU', 'vo', 'kz', 'Ġ#', 'SomosTodos', 'Dilma', 'Ġ#', 'Ray', 'nn', 'iere', 'Ġ#', 'Bu', 'y', 'Ray', 'nn', 'iere', 'On', 'I', 'tun', '</s>']}
tokens
{'logits': [-1.6449451446533203, -2.665390729904175, 4.34192419052124], 'probs': [0.0025029622483998537, 0.0009021537262015045, 0.9965948462486267], 'token_ids': [0, 597, 283, 274, 14837, 17724, 25170, 532, 16, 2533, 857, 7875, 836, 12231, 836, 2755, 274, 7103, 341, 329,

tokens
{'logits': [-3.496389389038086, 6.36854887008667, -2.5350778102874756], 'probs': [5.1955310482298955e-05, 0.9998121857643127, 0.00013586970453616232], 'token_ids': [0, 357, 12832, 30, 1996, 12433, 12341, 16, 1265, 274, 8543, 2656, 1507, 305, 144922, 341, 329, 88, 18, 265, 19, 85, 2956, 31619, 1043, 60, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 'Negativo', 'tokens': ['<s>', 'Ġ@', 'VEJA', ':', 'ĠEm', 'Ġcalamidade', 'Ġfinanceira', ',', 'Ġgoverno', 'Ġde', 'ĠMG', 'Ġcompra', 'Ġdois', 'Ġh', 'elicÃ³pteros', 'Ġhttps', '://', 't', '.', 'co', '/', 'q', 'af', 'iU', 'ip', 'X', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']}
tokens
{'logits': [6.492689609527588, -2.2986764907836914, -3.1829965114593506], 'probs': [0.9997852444648743, 0.00015200738562271, 6.277833017520607e-05], 'token_ids': [0, 274, 7103, 5990, 101034, 15814, 384, 8428, 13815, 356, 128537, 6475, 836, 341, 329, 88, 18, 265, 19, 9769, 57, 846, 4518, 39, 26, 2, 1, 1, 1, 1, 1], 

tokens
{'logits': [-3.2739901542663574, -2.992088556289673, 6.165467739105225], 'probs': [7.95088053564541e-05, 0.00010540069342823699, 0.9998151659965515], 'token_ids': [0, 15072, 5134, 18780, 3217, 389, 7481, 96411, 16, 7103, 11296, 1341, 19, 357, 51474, 67, 13756, 1970, 357, 113686, 570, 818, 75, 357, 15714, 38446, 89041, 357, 15325, 20966, 4915, 341, 329, 88, 18, 265, 19, 6890, 24, 61, 23, 75, 21, 2], 'label': 'Neutro', 'tokens': ['<s>', "'m", 'Ġat', 'ĠBig', 'ĠMais', 'Ġin', 'ĠGovernador', 'ĠValadares', ',', 'ĠMinas', 'ĠGerais', 'Ġw', '/', 'Ġ@', 'tue', '_', 'nunes', '07', 'Ġ@', 'comba', 'ten', 'tem', 'g', 'Ġ@', 'cle', 'verson', '0505', 'Ġ@', 'new', 'rand', 'ley', 'Ġhttps', '://', 't', '.', 'co', '/', 'sr', '4', 'Y', '3', 'g', '1', '</s>']}
tokens
{'logits': [6.97915506362915, -2.450558662414551, -3.0967235565185547], 'probs': [0.9998775720596313, 8.029231685213745e-05, 4.207736492389813e-05], 'token_ids': [0, 52, 118823, 25041, 836, 51253, 348, 3998, 5085, 36543, 54232, 29706, 274, 

In [7]:
predict(document_path='test_submission.json', model_path='snli-roberta-parameters-full/model.tar.gz', out_path='submissions_snli_roberta_parameters_full.csv')

Loading model from snli-roberta-parameters-full/model.tar.gz
Loading batches from test_submission.json for prediction
Predicted 100 sentences
Predicted 200 sentences
Predicted 300 sentences
Predicted 400 sentences
Predicted 500 sentences
Predicted 600 sentences
Predicted 700 sentences
Predicted 800 sentences
Predicted 900 sentences
Predicted 1000 sentences
Predicted 1100 sentences
Predicted 1200 sentences
Predicted 1300 sentences
Predicted 1400 sentences
Predicted 1500 sentences
Predicted 1600 sentences
Finished predicting 1640 sentences
Results saved in /media/discoD/repositorios/deeplearningufg/nlp/competicao1/submissions_snli_roberta_parameters_full.csv


In [2]:
stopwords = [word.strip() for word in open('stopwords-pt.txt', mode='r', encoding='utf8')]

def load_dataset(filename, train=True):
    raw_df = pd.read_csv(filename)
    if train:
        raw_df = raw_df.rename(columns={'Classificacao': 'label'})
    return raw_df

def describe_dataset(dataframe):
    print(dataframe.label.unique())
    print(dataframe.label.describe())
    print(dataframe.groupby('label')['Text'].count())

def write_predictions(dataframe, predictions, out_path):
    count = 0

    with open(out_path, mode='w', encoding='utf-8') as out_file:
        print('Saving predictions to %s' % out_path)
        out_file.write('Id,Category\n')
        idx = 0
        for result, (idx, row) in zip(predictions, dataframe.iterrows()):
            count += 1
            out_file.write('{},{}\n'.format(row['Id'], result))
            idx += 1
            if count % 100 == 0:
                print('Predicted %d sentences' % count)
    out_file.close()
    print('Finished predicting %d sentences' % count)
    print('Results saved in %s' % Path(out_path).absolute())
    
def train_svm_model(train_df, test_df = None, submission_df = None, submission_name: str = None):
    processed_clf_svm = Pipeline([('vect', CountVectorizer(stop_words=stopwords)), ('tfidf', TfidfTransformer()),
                                  ('clf-svm', SGDClassifier(loss='hinge', max_iter=2000, tol=1e-5, random_state=42))])

    processed_clf_svm = processed_clf_svm.fit(train_df['Text'], train_df['label'])
    if test_df is not None:
        predicted_svm = processed_clf_svm.predict(test_df['Text'])
        print(balanced_accuracy_score(test_df['label'], predicted_svm))
    if submission_name is not None:
        predictions = processed_clf_svm.predict(submission_df['Text'])
        write_predictions(submission_df, predictions, 'submissions_' + submission_name + '.csv')
    return processed_clf_svm, predictions

In [3]:
train_pd = load_dataset('train.csv')
test_pd = load_dataset('test.csv')

In [4]:
train_pd

,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,label,Observação,Id
0,Mon Jan 09 15:27:43 +0000 2017,Dois são detidos ao tentar jogar celulares e d...,NaN,NaN,NaN,Michele #beta #sdv,michelexmbeta,0,Positivo,NaN,6272
1,Sun Jan 08 02:14:34 +0000 2017,me matan esas minas q cambian 554 veces su fot...,NaN,NaN,Núñez - C.A.B.A.,Gaby Messina,gabymessina36,0,Neutro,NaN,1644
2,Sat Feb 11 09:49:11 +0000 2017,Líderes de motim em presídio de Minas Gerais s...,NaN,NaN,"Hollywood, CA",Wendie Rower,Wendie_Rower,0,Positivo,NaN,7956
3,Thu Jan 05 14:43:03 +0000 2017,#Mídia: Press Release from Business Wire : Di...,NaN,NaN,SP,Marcello Binder,binderbr,0,Neutro,NaN,85
4,Wed Feb 08 22:52:10 +0000 2017,Vacinação contra febre amarela é intensificada...,NaN,NaN,NaN,fodido,eufodeu,0,Positivo,NaN,6006
...,...,...,...,...,...,...,...,...,...,...,...
6554,Thu Jan 26 14:31:45 +0000 2017,Rio faz bloqueio contra febre amarela em munic...,NaN,NaN,Goiás - Brasil,Altair Tavares,altairtavares,0,Positivo,NaN,5735
6555,Fri Feb 10 18:13:01 +0000 2017,Governador Fernando Pimentel entrega 401 veícu...,NaN,NaN,Santos Dumont - MG -Brasil,POSSANTE ON LINE,possanteonline,0,Positivo,NaN,5192
6556,Thu Jan 05 17:19:20 +0000 2017,Secretaria de Educação faz reformulações para ...,NaN,NaN,Minas Gerais - Brasil,Uberlândia,PrefeituraUdia,0,Positivo,NaN,5391
6557,Thu Jan 05 14:15:02 +0000 2017,E governo ainda quer indenizar a família dos b...,NaN,NaN,NaN,Graça Azeredo,azeredo_mg,0,Neutro,NaN,861


In [5]:
describe_dataset(train_pd)

['Positivo' 'Neutro' 'Negativo']
count         6559
unique           3
top       Positivo
freq          2639
Name: label, dtype: object
label
Negativo    1970
Neutro      1950
Positivo    2639
Name: Text, dtype: int64


In [6]:
train_split_pd, test_split_pd = train_test_split(train_pd, stratify=train_pd['label'], test_size=0.2)

In [12]:
# train_split_pd[['Text', 'label']].rename(columns={'Text': 'text'}).to_json('train.json', orient='records', lines=True)
# test_split_pd[['Text', 'label']].rename(columns={'Text': 'text'}).to_json('test.json', orient='records', lines=True)
train_pd[['Text', 'label']].rename(columns={'Text': 'text'}).to_json('train_full.json', orient='records', lines=True)

In [29]:
describe_dataset(train_split_pd)

['Neutro' 'Positivo' 'Negativo']
count         5247
unique           3
top       Positivo
freq          2111
Name: label, dtype: object
label
Negativo    1576
Neutro      1560
Positivo    2111
Name: Text, dtype: int64


In [30]:
describe_dataset(test_split_pd)

['Negativo' 'Positivo' 'Neutro']
count         1312
unique           3
top       Positivo
freq           528
Name: label, dtype: object
label
Negativo    394
Neutro      390
Positivo    528
Name: Text, dtype: int64


In [55]:
model, predictions = train_svm_model(train_split_pd, test_split_pd, test_pd, 'stopwords_svm_tfidf')

0.9515849632600902
Saving predictions to submissions_stopwords_svm_tfidf.csv
Predicted 100 sentences
Predicted 200 sentences
Predicted 300 sentences
Predicted 400 sentences
Predicted 500 sentences
Predicted 600 sentences
Predicted 700 sentences
Predicted 800 sentences
Predicted 900 sentences
Predicted 1000 sentences
Predicted 1100 sentences
Predicted 1200 sentences
Predicted 1300 sentences
Predicted 1400 sentences
Predicted 1500 sentences
Predicted 1600 sentences
Finished predicting 1640 sentences
Results saved in /media/discoD/repositorios/deeplearningufg/nlp/competicao1/submissions_stopwords_svm_tfidf.csv


In [59]:
model, predictions = train_svm_model(train_df=train_pd, submission_df=test_pd, submission_name='stopwords_svm_tfidf_full')

Saving predictions to submissions_stopwords_svm_tfidf_full.csv
Predicted 100 sentences
Predicted 200 sentences
Predicted 300 sentences
Predicted 400 sentences
Predicted 500 sentences
Predicted 600 sentences
Predicted 700 sentences
Predicted 800 sentences
Predicted 900 sentences
Predicted 1000 sentences
Predicted 1100 sentences
Predicted 1200 sentences
Predicted 1300 sentences
Predicted 1400 sentences
Predicted 1500 sentences
Predicted 1600 sentences
Finished predicting 1640 sentences
Results saved in /media/discoD/repositorios/deeplearningufg/nlp/competicao1/submissions_stopwords_svm_tfidf_full.csv


In [5]:
test_pd

,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Observação,Id
0,Thu Jan 05 12:00:34 +0000 2017,RT @JDanieldf: Pedindo para que MG reaja? Reag...,NaN,NaN,Balneário Camboriú - SC,Mirela Franz,MiLick74,27,NaN,3568
1,Fri Jan 06 11:54:50 +0000 2017,Homem que matou ex-mulher e jogou corpo em cis...,NaN,NaN,Belo Horizonte MG Brasil,Ricardo Carlini,carlinibh,1,NaN,1323
2,Sat Feb 11 15:51:14 +0000 2017,"New post: ""Três adolescentes são apreendidos p...",NaN,NaN,NaN,Camila Maciel Serrão,CamilaMacielSer,0,NaN,7976
3,Wed Jan 04 18:08:43 +0000 2017,RT @AnaPaulaVolei: Mais 2 helicópteros!!A cara...,NaN,NaN,Sao Paulo,CLAUDIA DELAFIORI,cdelafiori,444,NaN,2408
4,Wed Jan 04 18:12:12 +0000 2017,"RT @UOLNoticias: Custaram R$ 21,8 milhões: Mes...",NaN,NaN,Jaboatão dos Guararapes,Rodrigo Calabria,CalabriaRodrigo,141,NaN,4435
...,...,...,...,...,...,...,...,...,...,...
1635,Sat Jan 07 12:51:37 +0000 2017,RT @ivo123zarate3: Me hace mal ver en instagra...,NaN,NaN,M e r c e d e s (Ctes),Ere Esse♡,RamoohSilvero,12,NaN,3536
1636,Wed Jan 25 14:18:35 +0000 2017,@PMMG190 - Militares da 22ª Cia prendem autore...,NaN,NaN,Minas Gerais,POLÍCIA MILITAR MG,pmmg190,0,NaN,6881
1637,Mon Jan 09 11:18:11 +0000 2017,Cadeia em Manaus tem 4 mortos; Estados pedem a...,NaN,NaN,NaN,Lenilda Miranda,nilda_ap,0,NaN,627
1638,Fri Jan 06 13:02:26 +0000 2017,Reforma da Previdência será feita no primeiro ...,NaN,NaN,Brasil,marli silvera dziadz,marlidzdz59,0,NaN,2165
